# Análise Exploratória: Incompatibilidades nos Dados

**Objetivo**: Identificar todos os casos de incompatibilidades nos dados de absentismo para discussão com RH.

**Abordagem**: Trabalhar ao nível de **nivel_4** (12 categorias) em vez de códigos individuais (40).

## Por que nivel_4?

- **Códigos individuais**: 40 códigos = 1600 combinações ❌ Impraticável!
- **nivel_4**: 12 categorias = **144 combinações** ✅ Manejável!

## Regras de Compatibilidade (nivel_4):

### ✅ Compatíveis (apenas 3 pares):
1. **Atraso** + **Presença Completa** ✓ (chegou atrasado mas trabalhou)
2. **Atraso** + **Exame Escolar** ✓ (exame + chegou atrasado)
3. **Presença Completa** + **Exame Escolar** ✓ (exame + trabalhou)

### ❌ Incompatíveis:
- **TODO o resto** → Incompatível com TUDO
- Exemplos: Férias + Trabalho, Baixa Médica + Presença, Falta + Presença, etc.

## Metodologia:

1. **Criar Matriz 12x12**: Baseada em nivel_4, pré-preenchida com regras
2. **Revisar e Ajustar**: Validar no Excel (muito mais rápido!)
3. **Testar Incompatibilidades**: Identificar casos reais nos dados
4. **Exportar Resultados**: Excel com detalhes para discussão com RH

---

## 1. Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Carregar códigos (mesma estrutura do notebook de apresentação)
print('Carregando códigos...')
codigos = pd.read_excel('códigos.xlsx', engine='openpyxl')
codigos.columns = ['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']

# Limpar espaços
for col in codigos.columns:
    if codigos[col].dtype == 'object':
        codigos[col] = codigos[col].str.strip()

print(f'✅ Códigos carregados: {len(codigos)} códigos')
print(f'   Níveis: N1={codigos["nivel_1"].nunique()}, N2={codigos["nivel_2"].nunique()}, N3={codigos["nivel_3"].nunique()}, N4={codigos["nivel_4"].nunique()}')

# Carregar dataset
print('\nCarregando dataset...')
df_raw = pd.read_csv('combined_data.csv', sep=',', parse_dates=['Data'])

# Merge com códigos para ter todos os níveis
df_raw = df_raw.merge(
    codigos[['codigo', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']], 
    left_on='segmento_processado_codigo', 
    right_on='codigo', 
    how='left'
)
df_raw.drop('codigo', axis=1, inplace=True)

print(f'\n✅ Dataset carregado: {len(df_raw):,} registros')
print(f'   Período: {df_raw["Data"].min().strftime("%Y-%m-%d")} a {df_raw["Data"].max().strftime("%Y-%m-%d")}')
print(f'   Colaboradores únicos: {df_raw["login_colaborador"].nunique():,}')
print(f'   Colunas com níveis: {[c for c in df_raw.columns if "nivel" in c]}')

---

## 2. Criação da Matriz de Compatibilidade (nivel_4)

In [ ]:
# Obter lista de categorias únicas do nivel_4
lista_nivel4 = sorted(codigos['nivel_4'].unique())
n_categorias = len(lista_nivel4)

print('='*90)
print('📋 CRIAÇÃO DA MATRIZ DE COMPATIBILIDADE (NIVEL_4)')
print('='*90)
print(f'\nTotal de categorias (nivel_4): {n_categorias}')
print(f'Tamanho da matriz: {n_categorias}x{n_categorias} = {n_categorias*n_categorias} combinações')
print(f'\n📋 Categorias do nivel_4:')
for i, cat in enumerate(lista_nivel4, 1):
    # Contar quantos códigos pertencem a esta categoria
    n_codigos_cat = (codigos['nivel_4'] == cat).sum()
    print(f'   {i:2d}. {cat:30s} ({n_codigos_cat} códigos)')

# Criar matriz de compatibilidade
# Por padrão: TUDO incompatível
matriz_compat = pd.DataFrame(
    'Incompatível',  # Valor padrão: tudo incompatível!
    index=lista_nivel4,
    columns=lista_nivel4
)

print('\n' + '='*90)
print('Aplicando regras de compatibilidade...')
print('='*90)

# REGRA: Diagonal (categoria consigo mesma) é sempre compatível
for cat in lista_nivel4:
    matriz_compat.loc[cat, cat] = 'Compatível'

# REGRA: Apenas 3 pares são compatíveis (além da diagonal)
pares_compativeis = [
    ('Atraso', 'Presença Completa'),
    ('Atraso', 'Exame Escolar'),
    ('Presença Completa', 'Exame Escolar')
]

for cat1, cat2 in pares_compativeis:
    if cat1 in matriz_compat.index and cat2 in matriz_compat.columns:
        matriz_compat.loc[cat1, cat2] = 'Compatível'
        matriz_compat.loc[cat2, cat1] = 'Compatível'  # Simétrica

# Contar compatibilidades
total_pares = n_categorias * (n_categorias - 1) // 2  # Sem contar diagonal
pares_compativeis_count = (matriz_compat == 'Compatível').sum().sum() - n_categorias  # Subtrair diagonal
pares_compativeis_count = pares_compativeis_count // 2  # Dividir por 2 (simétrica)

print(f'\n✅ Matriz criada!')
print(f'   Total de pares (sem diagonal): {total_pares}')
print(f'   Pares Compatíveis: {pares_compativeis_count} ({pares_compativeis_count/total_pares*100:.1f}%)')
print(f'   Pares Incompatíveis: {total_pares - pares_compativeis_count} ({(total_pares-pares_compativeis_count)/total_pares*100:.1f}%)')

print(f'\n💡 Regras aplicadas:')
print(f'   ✅ Categoria consigo mesma: Compatível (diagonal)')
print(f'   ✅ Atraso + Presença Completa: Compatível')
print(f'   ✅ Atraso + Exame Escolar: Compatível')
print(f'   ✅ Presença Completa + Exame Escolar: Compatível')
print(f'   ❌ TODO o resto: Incompatível')

print(f'\n📝 Próximo passo: Exportar matriz para Excel e revisar')

---

## 3. Exportação da Matriz para Excel

In [ ]:
# Exportar matriz para Excel
output_matriz = 'matriz_compatibilidade_nivel4.xlsx'

with pd.ExcelWriter(output_matriz, engine='openpyxl') as writer:
    # Exportar matriz
    matriz_compat.to_excel(writer, sheet_name='Matriz_Compatibilidade', index=True)
    
    # Criar folha de referência: nivel_4 -> códigos
    df_nivel4_ref = codigos.groupby('nivel_4').agg({
        'codigo': lambda x: ', '.join(sorted(x)),
        'descricao': lambda x: f"{len(x)} códigos"
    }).reset_index()
    df_nivel4_ref.columns = ['Categoria_nivel_4', 'Códigos', 'Total']
    df_nivel4_ref = df_nivel4_ref.sort_values('Categoria_nivel_4')
    df_nivel4_ref.to_excel(writer, sheet_name='Referência_nivel_4', index=False)
    
    # Criar folha de instruções
    instrucoes = pd.DataFrame({
        'Instruções': [
            '🎯 OBJETIVO: Validar regras de compatibilidade ao nível de nivel_4',
            '',
            '✅ MATRIZ 12x12 (144 combinações) - Muito mais manejável que 40x40!',
            '',
            '📋 REGRAS PRÉ-APLICADAS:',
            '   ✅ Categoria consigo mesma → Compatível (diagonal)',
            '   ✅ Atraso + Presença Completa → Compatível',
            '   ✅ Atraso + Exame Escolar → Compatível',
            '   ✅ Presença Completa + Exame Escolar → Compatível',
            '   ❌ TODO o resto → Incompatível',
            '',
            '💡 LÓGICA DAS REGRAS:',
            '   - Podes chegar atrasado e trabalhar ✓',
            '   - Podes ter exame escolar e trabalhar ✓',
            '   - Podes ter exame e chegar atrasado ✓',
            '   - MAS não podes ter férias e trabalhar ✗',
            '   - MAS não podes ter baixa médica e trabalhar ✗',
            '   - MAS não podes ter falta e trabalhar ✗',
            '',
            '📝 O QUE FAZER:',
            '   1. Revisar folha "Matriz_Compatibilidade" (12x12)',
            '   2. Verificar se as 3 compatibilidades fazem sentido',
            '   3. Se necessário, AJUSTAR (adicionar ou remover "Compatível")',
            '   4. Folha "Referência_nivel_4" mostra quais códigos estão em cada categoria',
            '',
            '⚠️ IMPORTANTE:',
            '   - Não mudar nome do ficheiro nem das folhas',
            '   - Usar apenas "Compatível" ou "Incompatível"',
            '   - Após editar, GUARDAR e executar Secção 4 do notebook',
            '',
            f'📊 ESTATÍSTICAS:',
            f'   - Categorias (nivel_4): {n_categorias}',
            f'   - Total de pares: {total_pares}',
            f'   - Compatíveis: {pares_compativeis_count} ({pares_compativeis_count/total_pares*100:.1f}%)',
            f'   - Incompatíveis: {total_pares - pares_compativeis_count} ({(total_pares-pares_compativeis_count)/total_pares*100:.1f}%)'
        ]
    })
    instrucoes.to_excel(writer, sheet_name='INSTRUÇÕES', index=False)
    
    # Lista dos pares compatíveis (fácil de revisar)
    pares_compat_lista = []
    for i, cat1 in enumerate(lista_nivel4):
        for j, cat2 in enumerate(lista_nivel4):
            if i < j and matriz_compat.loc[cat1, cat2] == 'Compatível':
                codigos_cat1 = codigos[codigos['nivel_4'] == cat1]['codigo'].tolist()
                codigos_cat2 = codigos[codigos['nivel_4'] == cat2]['codigo'].tolist()
                pares_compat_lista.append({
                    'Categoria 1': cat1,
                    'Códigos em Cat1': ', '.join(codigos_cat1),
                    'Categoria 2': cat2,
                    'Códigos em Cat2': ', '.join(codigos_cat2),
                    'Regra': 'Automática'
                })
    
    df_compat = pd.DataFrame(pares_compat_lista)
    df_compat.to_excel(writer, sheet_name='Lista_Compatíveis', index=False)

print(f'✅ Matriz exportada: {output_matriz}')
print(f'\n📋 Folhas criadas:')
print(f'   1. INSTRUÇÕES: Como usar e validar')
print(f'   2. Matriz_Compatibilidade: Matriz 12x12 PRÉ-PREENCHIDA')
print(f'   3. Lista_Compatíveis: Lista dos {pares_compativeis_count} pares compatíveis')
print(f'   4. Referência_nivel_4: Mapeamento nivel_4 → códigos')
print(f'\n🎯 PRÓXIMO PASSO:')
print(f'   1. Abrir: {output_matriz}')
print(f'   2. Revisar folha "Lista_Compatíveis" (só {pares_compativeis_count} pares!)')
print(f'   3. Se necessário, ajustar na "Matriz_Compatibilidade"')
print(f'   4. Guardar ficheiro')
print(f'   5. Executar Secção 4 do notebook')

---

## 4. Teste de Incompatibilidades (executar APÓS editar a matriz)

**⚠️ ATENÇÃO**: Só executar esta secção DEPOIS de editar e guardar `matriz_compatibilidade_nivel4.xlsx`

In [ ]:
# Carregar matriz editada
print('Carregando matriz de compatibilidade editada...')
matriz_editada = pd.read_excel(
    'matriz_compatibilidade_nivel4.xlsx',
    sheet_name='Matriz_Compatibilidade',
    index_col=0
)

# Contar incompatibilidades definidas
total_incompativeis = (matriz_editada == 'Incompatível').sum().sum()
total_compativeis = (matriz_editada == 'Compatível').sum().sum()

# Subtrair diagonal e dividir por 2 (simétrica)
pares_incompativeis_unicos = (total_incompativeis - n_categorias) // 2
pares_compativeis_unicos = (total_compativeis - n_categorias) // 2

print(f'✅ Matriz carregada')
print(f'   Tamanho: {matriz_editada.shape[0]}x{matriz_editada.shape[1]}')
print(f'   Pares Compatíveis: {pares_compativeis_unicos}')
print(f'   Pares Incompatíveis: {pares_incompativeis_unicos}')

# Identificar dias com múltiplos registros
print('\nIdentificando dias com múltiplos registros...')
registros_por_dia = df_raw.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = registros_por_dia[registros_por_dia > 1]

print(f'✅ Total de dias-colaborador únicos: {len(registros_por_dia):,}')
print(f'   Dias com MÚLTIPLOS registros: {len(dias_duplicados):,} ({len(dias_duplicados)/len(registros_por_dia)*100:.1f}%)')

# Filtrar apenas dias duplicados
dias_dup_list = dias_duplicados.index.tolist()
df_dup = df_raw.set_index(['login_colaborador', 'Data']).loc[dias_dup_list].reset_index()

print(f'   Dataset de dias duplicados: {len(df_dup):,} registros')

### 4.1. Teste de Todas as Incompatibilidades

In [ ]:
print('='*90)
print('🚨 TESTANDO INCOMPATIBILIDADES (NIVEL_4) - VERSÃO OTIMIZADA')
print('='*90)

# Preparar dados para teste eficiente
print('Preparando dados para teste eficiente...')

# Criar DataFrame agrupado com todas as informações necessárias
# Isto evita filtragens repetidas dentro do loop
df_dup_grouped = df_dup.groupby(['login_colaborador', 'Data']).apply(
    lambda g: pd.Series({
        'nome_colaborador': g['nome_colaborador'].iloc[0],
        'categorias_nivel4': list(g['nivel_4'].unique()),
        'dados_completos': g[['nivel_4', 'segmento_processado_codigo']].to_dict('records')
    })
).reset_index()

print(f'✅ Dados preparados: {len(df_dup_grouped):,} dias únicos a testar')

# Dicionário para guardar resultados
resultados_incompatibilidades = {}
total_incompatibilidades_encontradas = 0

# Processar cada dia
print('\nTestando incompatibilidades...')
for idx, row in df_dup_grouped.iterrows():
    if idx % 5000 == 0 and idx > 0:
        print(f'   Processados {idx:,}/{len(df_dup_grouped):,} dias...')
    
    login = row['login_colaborador']
    data = row['Data']
    nome = row['nome_colaborador']
    categorias_dia = row['categorias_nivel4']
    dados_completos = row['dados_completos']
    
    # Testar todos os pares de categorias nivel_4
    for i, cat1 in enumerate(categorias_dia):
        for cat2 in categorias_dia[i+1:]:
            # Verificar na matriz se são incompatíveis
            if cat1 in matriz_editada.index and cat2 in matriz_editada.columns:
                compatibilidade = matriz_editada.loc[cat1, cat2]
                
                if compatibilidade == 'Incompatível':
                    total_incompatibilidades_encontradas += 1
                    
                    # Criar chave única para este par
                    par = tuple(sorted([cat1, cat2]))
                    
                    # Inicializar lista se necessário
                    if par not in resultados_incompatibilidades:
                        resultados_incompatibilidades[par] = []
                    
                    # Obter códigos específicos - filtrar da lista já preparada
                    codigos_cat1 = [d['segmento_processado_codigo'] for d in dados_completos if d['nivel_4'] == cat1]
                    codigos_cat2 = [d['segmento_processado_codigo'] for d in dados_completos if d['nivel_4'] == cat2]
                    
                    resultados_incompatibilidades[par].append({
                        'Data': data,
                        'Login': login,
                        'Nome': nome,
                        'Categoria_1': cat1,
                        'Códigos_Cat1': ', '.join(codigos_cat1),
                        'Categoria_2': cat2,
                        'Códigos_Cat2': ', '.join(codigos_cat2)
                    })

print(f'\n✅ Teste concluído!')
print(f'   Dias testados: {len(df_dup_grouped):,}')
print(f'   Incompatibilidades encontradas: {total_incompatibilidades_encontradas:,}')
print(f'   Pares de categorias incompatíveis encontrados: {len(resultados_incompatibilidades)}')

# Mostrar resumo dos pares incompatíveis encontrados
if len(resultados_incompatibilidades) > 0:
    print(f'\n📋 RESUMO DOS PARES INCOMPATÍVEIS ENCONTRADOS:')
    for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
        cat1, cat2 = par
        print(f'   🔴 {cat1} + {cat2}: {len(casos)} casos')
else:
    print(f'\n✅ Nenhuma incompatibilidade encontrada nos dados!')

---

## 5. Exportação dos Resultados

In [ ]:
if len(resultados_incompatibilidades) == 0:
    print('⚠️ Nenhuma incompatibilidade encontrada. Nada para exportar.')
else:
    # Criar Excel com uma folha por par de categorias incompatíveis
    output_file = 'incompatibilidades_encontradas.xlsx'
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Criar folha de resumo
        resumo_data = []
        for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
            cat1, cat2 = par
            resumo_data.append({
                'Categoria 1 (nivel_4)': cat1,
                'Categoria 2 (nivel_4)': cat2,
                'Casos Encontrados': len(casos),
                'Folha': f'{cat1[:15]}+{cat2[:15]}'
            })
        
        df_resumo = pd.DataFrame(resumo_data)
        df_resumo.to_excel(writer, sheet_name='RESUMO', index=False)
        
        # Criar uma folha por par incompatível
        for par, casos in resultados_incompatibilidades.items():
            cat1, cat2 = par
            
            # Criar DataFrame com os casos
            df_casos = pd.DataFrame(casos)
            
            # Reordenar colunas
            df_casos = df_casos[['Data', 'Login', 'Nome', 'Categoria_1', 'Códigos_Cat1', 'Categoria_2', 'Códigos_Cat2']]
            
            # Nome da folha (máximo 31 caracteres)
            sheet_name = f'{cat1[:15]}+{cat2[:15]}'[:31]
            
            df_casos.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f'✅ Resultados exportados: {output_file}')
    print(f'\n📋 Estrutura do ficheiro:')
    print(f'   - Folha RESUMO: {len(resultados_incompatibilidades)} pares incompatíveis (nivel_4)')
    print(f'   - {len(resultados_incompatibilidades)} folhas com detalhes de cada par')
    print(f'   - Cada caso inclui: Data, Login, Nome, Categorias nivel_4, Códigos específicos')
    print(f'\n📊 Total de casos: {total_incompatibilidades_encontradas:,}')
    print(f'\n🎯 Próximo passo: Analisar com RH e decidir ação para cada tipo de incompatibilidade')

---

## Resumo do Processo

### Workflow Completo:

1. ✅ **Carregamento de Dados**: Códigos e dataset
2. ✅ **Criação da Matriz 12x12**: Baseada em nivel_4 (não códigos individuais!)
3. ✅ **Exportação da Matriz**: `matriz_compatibilidade_nivel4.xlsx`
4. 📝 **Revisão Manual**: Validar 3 pares compatíveis (muito rápido!)
5. 🔍 **Teste de Incompatibilidades**: Identificar todos os casos reais nos dados
6. 📊 **Exportação de Resultados**: `incompatibilidades_encontradas.xlsx`

### Por que nivel_4 e não códigos?

- **Códigos individuais**: 40 códigos = 1600 combinações ❌ Impraticável!
- **nivel_4**: 12 categorias = **144 combinações** ✅ Manejável!
- **Regras simples**: Apenas 3 pares compatíveis (Atraso/Presença/Exame)
- **Fácil de revisar**: 5 minutos vs vários dias

### Vantagens desta Abordagem:

- ✅ **Pragmático**: 144 combinações em vez de 1600
- ✅ **Regras claras**: Apenas 3 pares compatíveis, resto incompatível
- ✅ **Rápido de validar**: Lista de 3 compatibilidades para revisar
- ✅ **Flexível**: Fácil ajustar se necessário (editar Excel)
- ✅ **Auditável**: Matriz explícita de todas as regras
- ✅ **Reutilizável**: Matriz pode ser usada em análises futuras

### Ficheiros Gerados:

1. **matriz_compatibilidade_nivel4.xlsx**: Matriz 12x12 para validar
2. **incompatibilidades_encontradas.xlsx**: Resultados com casos reais